<a href="https://colab.research.google.com/github/barnabyashford/llm-powered-social-listener/blob/main/TestsetGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LLM Project: Test data creation pipeline**

In [ ]:
%pip install tqdm gtts playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=3ff84967955ff156dfe5912fc5ac7f76f29405735918721ec84609518afae8fa
  Stored in directory: /root/.cache/pip/wheels/50/98/42/62753a9e1fb97579a0ce2f84f7db4c21c09d03bb2091e6cef4
Successfully built playsound


In [ ]:
import json
from gtts import gTTS
from playsound import playsound
from itertools import islice
import os
from datetime import datetime
import pytz
import pandas as pd
import re
from tqdm.notebook import tqdm
import pprint

In [1]:
import google.generativeai as genai
from google.genai import types
from google.colab import userdata
import pandas as pd
import time
from tqdm.notebook import tqdm
from google.api_core import retry

# GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
SPARE_GEMINI_API_KEY = userdata.get('SPARE_GEMINI_API_KEY')

# genai.configure(api_key=GEMINI_API_KEY)
genai.configure(api_key=SPARE_GEMINI_API_KEY)
generation_config = genai.GenerationConfig(
        temperature=0,
)
llm = genai.GenerativeModel("gemini-2.0-flash", generation_config=generation_config)

# Catch transient Gemini errors.
def is_retryable(e) -> bool:
    if retry.if_transient_error(e):
        # Good practice, but probably won't fire with the google-genai SDK
        return True
    elif (isinstance(e, genai.errors.ClientError) and e.code == 429):
        # Catch 429 quota exceeded errors
        return True
    elif (isinstance(e, genai.errors.ServerError) and e.code == 503):
        # Catch 503 model overloaded errors
        return True
    else:
        return False

@retry.Retry(predicate=is_retryable)
def generate_content_with_rate_limit(prompt):
  return llm.generate_content(prompt).text

In [ ]:
submission_path = r"C:\Users\growt\Downloads\reddit\submissions\RS_2018-10" # submissions
comment_path = r"C:\Users\growt\Downloads\reddit\comments\RC_2018-10" # comments

save_dir = r"C:\Users\growt\OneDrive\เดสก์ท็อป\UFCLESGOO"

In [ ]:
def play_finish_sound():
  finished = gTTS(text="kansei itashimashita", lang="ja", slow=False)
  finished.save("finished.mp3")
  playsound("finished.mp3")
  os.remove("finished.mp3")

## **Submission filtering**

Here we filter all submissions from `r/ufc` in the month of October 2018 for submissions mentioning Khabib and Conor McGregor using simple keyword search. As we need opinion of people ***after*** the match, we also need to filter, not only on submission text `selftext`, but also `created_utc` as well.

In [ ]:
save_filename = "ufc_submissions.jsonl"

target_datetime_utc = datetime(2018, 10, 7, 4, 30, 0, tzinfo=pytz.utc) # 2018-10-07 04:30:00 UTC, the end of the UFC 229 fight
target_timestamp = target_datetime_utc.timestamp()

print(f"Target timestamp: {target_timestamp} : {target_datetime_utc}")

after_target_datetime_count = 0

Target timestamp: 1538886600.0 : 2018-10-07 04:30:00+00:00


In [ ]:
save_file = open(os.path.join(save_dir, save_filename), mode="w", encoding="utf-8")
ufc_sub_count = 0

with open(submission_path, mode="r", encoding="utf-8") as data:
    for line in tqdm(data, desc="Processing submissions"): # used tqdm.tqdm, hence no progress bar
        try:
            line = json.loads(line)
            if line["subreddit"] == "ufc" and "khabib" in line["selftext"].lower() and ("conor" in line["selftext"].lower() or "mcgregor" in line["selftext"].lower()) and line["created_utc"] > target_timestamp:
                print("found ufc submission about khabib vs conor match")
                ufc_sub_count += 1
                json.dump(line, save_file)
                save_file.write("\n") # for convenience, the last `\n` will be removed later.
                save_file.flush()
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue

print(f"Total relevant submissions found: {ufc_sub_count}")
save_file.close()
play_finish_sound() # play the sound when finished

Processing submissions: 2781129it [01:09, 38556.28it/s]

found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission

Processing submissions: 2788914it [01:10, 38106.61it/s]

found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match
found ufc submission about khabib vs conor match


Processing submissions: 2796715it [01:10, 38391.83it/s]

...

found ufc submission about khabib vs conor match


Processing submissions: 13433866it [05:43, 39982.03it/s]

found ufc submission about khabib vs conor match


Processing submissions: 13975028it [05:57, 39120.79it/s]


Total relevant submissions found: 200


In [ ]:
ufc_229_submissions = pd.read_json(os.path.join(save_dir, save_filename), lines=True)
display(ufc_229_submissions.info())

ids = ufc_229_submissions["id"].tolist()
print(ids)
print(f"Total submissions: {len(ids)}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 77 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   archived                       200 non-null    bool   
 1   author                         200 non-null    object 
 2   author_created_utc             197 non-null    float64
 3   author_flair_background_color  3 non-null      object 
 4   author_flair_css_class         0 non-null      float64
 5   author_flair_richtext          197 non-null    object 
 6   author_flair_template_id       0 non-null      float64
 7   author_flair_text              0 non-null      float64
 8   author_flair_text_color        3 non-null      object 
 9   author_flair_type              197 non-null    object 
 10  author_fullname                197 non-null    object 
 11  author_patreon_flair           197 non-null    float64
 12  can_gild                       200 non-null    boo

None

['9m2q9s', '9m2qxc', '9m2r8s', '9m2rct', '9m2rnc', '9m2rtu', '9m2rz9', '9m2s1c', '9m2s5c', '9m2shf', '9m2sqn', '9m2ssy', '9m2svh', '9m2sz2', '9m2taz', '9m2twr', '9m2tx9', '9m2u49', '9m2ufd', '9m2vk3', '9m2vl2', '9m2vp2', '9m2vyp', '9m2vza', '9m2w9e', '9m2wkt', '9m2x17', '9m2xnh', '9m2xr6', '9m2xra', '9m2xte', '9m2xu4', '9m2ycy', '9m2z5y', '9m2zgw', '9m2zzd', '9m30ho', '9m319p', '9m31q5', '9m321n', '9m322j', '9m32bg', '9m3312', '9m33db', '9m34ar', '9m34rz', '9m3am4', '9m3d35', '9m3evd', '9m3ftz', '9m3i5a', '9m3kr9', '9m3mha', '9m3o91', '9m3p6c', '9m3qny', '9m3sm4', '9m3wt5', '9m41iy', '9m41nv', '9m44y2', '9m47cq', '9m49gt', '9m4ab6', '9m4d4a', '9m4hj2', '9m4hqz', '9m4kcq', '9m4luj', '9m4ra1', '9m4uc5', '9m52tu', '9m57nt', '9m59mg', '9m5fgg', '9m5r66', '9m5um9', '9m5yyj', '9m60zi', '9m63b0', '9m63i5', '9m6ej6', '9m6fbp', '9m6fmq', '9m6gf7', '9m6hpn', '9m6rm8', '9m6sfl', '9m6vmw', '9m70vw', '9m7315', '9m7975', '9m7dty', '9m7hwz', '9m7rvo', '9m7v3b', '9m7wxz', '9m8786', '9m8ina', '9m8mij',

## **Comment filtering**

Using `id` attribute in submission files, we can filter each comment by matching their `link_id` (be sure to remove `t3_` prefix first) with target submission `id`.

In [ ]:
save_filename = "ufc_comments.jsonl"

comment_count_by_sub_id = {id: 0 for id in ids}

In [ ]:
with open(comment_path, mode="r", encoding="utf-8") as data:

  save_file = open(os.path.join(save_dir, save_filename), mode="w", encoding="utf-8")

  for line in tqdm(data, desc="Processing comments"):
    try:
      line = json.loads(line)
      if line["link_id"][3:] in comment_count_by_sub_id.keys():
        comment_count_by_sub_id[line["link_id"][3:]] += 1
        json.dump(line, save_file)
        save_file.write("\n") # for convenience, the last `\n` will be removed later.
        save_file.flush()
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        continue

  save_file.close()

print(f"Total relevant comments found: {sum(comment_count_by_sub_id.values())}")
play_finish_sound() # play the sound when finished

Processing comments: 0it [00:00, ?it/s]

Total relevant comments found: 5327


Here we filter to only comments that are not deleted nor removed.

In [ ]:
new_save_filename = "ufc_filtered_comments.jsonl"

comments_with_body_count = 0

comment_count_by_sub_id = {id: 0 for id in ids}

In [ ]:
filtered_comments = open(os.path.join(save_dir, new_save_filename), mode="w", encoding="utf-8")

with open(os.path.join(save_dir, save_filename), mode="r", encoding="utf-8") as data:
    for line in tqdm(data, desc="Filtering comments"):
        try:
            line = json.loads(line)
            if line["body"].lower() != "[deleted]" and line["body"].lower() != "[removed]":
                comment_count_by_sub_id[line["link_id"][3:]] += 1
                json.dump(line, filtered_comments)
                filtered_comments.write("\n") # for convenience, the last `\n` will be removed later.
                filtered_comments.flush()
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue

print(f"Total relevant comments with body found: {sum(comment_count_by_sub_id.values())}")
filtered_comments.close()

Filtering comments: 0it [00:00, ?it/s]

Total relevant comments with body found: 5023


In [ ]:
k = 40

submissions_with_more_than_k_comments = {id: count for id, count in comment_count_by_sub_id.items() if count > k}
print(f"Total submissions with more than {k} comments: {len(submissions_with_more_than_k_comments)}")

print(f"Total comments from filtered submissions: {sum(submissions_with_more_than_k_comments.values())}")

Total submissions with more than 40 comments: 23
Total comments from filtered submissions: 2668


In [ ]:
for id in submissions_with_more_than_k_comments.keys():
    print(f"Submission ID: {id}")
    print(f"Comment count: {submissions_with_more_than_k_comments[id]}")

Submission ID: 9m2rtu
Comment count: 788
Submission ID: 9m2taz
Comment count: 88
Submission ID: 9m2vp2
Comment count: 43
Submission ID: 9m2xra
Comment count: 57
Submission ID: 9m3312
Comment count: 71
Submission ID: 9m3sm4
Comment count: 47
Submission ID: 9m4hj2
Comment count: 112
Submission ID: 9m9yni
Comment count: 69
Submission ID: 9mbxvc
Comment count: 58
Submission ID: 9mfjom
Comment count: 222
Submission ID: 9mol6q
Comment count: 50
Submission ID: 9mpl3m
Comment count: 46
Submission ID: 9mpzu5
Comment count: 55
Submission ID: 9msgpy
Comment count: 227
Submission ID: 9mu3ty
Comment count: 199
Submission ID: 9mvlmx
Comment count: 42
Submission ID: 9mvysg
Comment count: 68
Submission ID: 9nnjsj
Comment count: 62
Submission ID: 9ns1ho
Comment count: 138
Submission ID: 9ns7o6
Comment count: 43
Submission ID: 9obu5r
Comment count: 56
Submission ID: 9omfiw
Comment count: 78
Submission ID: 9s8od9
Comment count: 49


## **Finalise data to use**

In [ ]:
final_submission_file = open(os.path.join(save_dir, "ufc_final_submissions.jsonl"), mode="w", encoding="utf-8")
final_comment_file = open(os.path.join(save_dir, "ufc_final_comments.jsonl"), mode="w", encoding="utf-8")

comments = pd.read_json(os.path.join(save_dir, new_save_filename), lines=True)

for id in tqdm(submissions_with_more_than_k_comments.keys(), desc="Processing final submissions"):
    submission = ufc_229_submissions[ufc_229_submissions["id"] == id].to_dict(orient="records")[0]
    json.dump(submission, final_submission_file)
    final_submission_file.write("\n")
    final_submission_file.flush()

    submission_comments = comments[comments["link_id"] == f"t3_{id}"]
    for _, comment in tqdm(submission_comments.iterrows(), desc=f"Processing comments for submission: {id}"):
        comment = comment.to_dict()
        json.dump(comment, final_comment_file)
        final_comment_file.write("\n")
        final_comment_file.flush()

Processing final submissions:   0%|          | 0/23 [00:00<?, ?it/s]

Processing comments for submission: 9m2rtu: 0it [00:00, ?it/s]

Processing comments for submission: 9m2taz: 0it [00:00, ?it/s]

Processing comments for submission: 9m2vp2: 0it [00:00, ?it/s]

Processing comments for submission: 9m2xra: 0it [00:00, ?it/s]

...

Processing comments for submission: 9ns1ho: 0it [00:00, ?it/s]

Processing comments for submission: 9ns7o6: 0it [00:00, ?it/s]

Processing comments for submission: 9obu5r: 0it [00:00, ?it/s]

Processing comments for submission: 9omfiw: 0it [00:00, ?it/s]

Processing comments for submission: 9s8od9: 0it [00:00, ?it/s]

## **Summary Generation**

## Skimmed to 20 posts

In [ ]:
# No longer available
# !wget https://raw.githubusercontent.com/barnabyashford/llm-powered-social-listener/refs/heads/main/ufc_final_submissions.jsonl -O submissions.jsonl
# !wget https://raw.githubusercontent.com/barnabyashford/llm-powered-social-listener/refs/heads/main/ufc_final_comments.jsonl -O comments.jsonl

--2025-04-21 04:36:53--  https://raw.githubusercontent.com/barnabyashford/llm-powered-social-listener/refs/heads/main/ufc_final_submissions.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70759 (69K) [text/plain]
Saving to: ‘submissions.jsonl’

submissions.jsonl   100%[===================>]  69.10K  --.-KB/s    in 0s      

2025-04-21 04:36:54 (136 MB/s) - ‘submissions.jsonl’ saved [70759/70759]

--2025-04-21 04:36:54--  https://raw.githubusercontent.com/barnabyashford/llm-powered-social-listener/refs/heads/main/ufc_final_comments.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conne

In [ ]:
cwd = os.getcwd()
submission_filepath = os.path.join(cwd, "submissions.jsonl")
comment_filepath = os.path.join(cwd, "comments.jsonl")

In [ ]:
submissions = pd.read_json(submission_filepath, lines=True)
comments = pd.read_json(comment_filepath, lines=True)

In [ ]:
sub_comm_count = {f"t3_{id}":0 for id in submissions["id"].tolist()}

In [ ]:
for _, comment in comments.iterrows():
  sub_comm_count[comment["link_id"]] += 1

In [ ]:
sub_comm_count

{'t3_9m2rtu': 788,
 't3_9m2taz': 88,
 't3_9m2vp2': 43,
 't3_9m2xra': 57,
 't3_9m3312': 71,
 't3_9m3sm4': 47,
 't3_9m4hj2': 112,
 't3_9m9yni': 69,
 't3_9mbxvc': 58,
 't3_9mfjom': 222,
 't3_9mol6q': 50,
 't3_9mpl3m': 46,
 't3_9mpzu5': 55,
 't3_9msgpy': 227,
 't3_9mu3ty': 199,
 't3_9mvlmx': 42,
 't3_9mvysg': 68,
 't3_9nnjsj': 62,
 't3_9ns1ho': 138,
 't3_9ns7o6': 43,
 't3_9obu5r': 56,
 't3_9omfiw': 78,
 't3_9s8od9': 49}

In [ ]:
least_twenty = {sub_id[3:]:count for sub_id, count in sub_comm_count.items() if count < 222}

In [ ]:
print(f"Total submissions: {len(least_twenty)}")
print(f"Total comments: {sum(least_twenty.values())}")

Total submissions: 20
Total comments: 1431


In [ ]:
least_twenty_submissions = submissions[submissions["id"].isin(least_twenty.keys())]
least_twenty_comments = comments[comments["link_id"].str[3:].isin(least_twenty.keys())]

In [ ]:
least_twenty_submissions.to_json("least_twenty_submissions.jsonl", orient="records", lines=True)
least_twenty_comments.to_json("least_twenty_comments.jsonl", orient="records", lines=True)

## Generation

In [2]:
!wget https://raw.githubusercontent.com/barnabyashford/llm-powered-social-listener/refs/heads/main/submissions.jsonl -O submissions.jsonl
!wget https://raw.githubusercontent.com/barnabyashford/llm-powered-social-listener/refs/heads/main/comments.jsonl -O comments.jsonl

--2025-04-21 19:20:25--  https://raw.githubusercontent.com/barnabyashford/llm-powered-social-listener/refs/heads/main/submissions.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60719 (59K) [text/plain]
Saving to: ‘submissions.jsonl’

submissions.jsonl   100%[===================>]  59.30K  --.-KB/s    in 0.01s   

2025-04-21 19:20:25 (4.25 MB/s) - ‘submissions.jsonl’ saved [60719/60719]

--2025-04-21 19:20:26--  https://raw.githubusercontent.com/barnabyashford/llm-powered-social-listener/refs/heads/main/comments.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request 

In [7]:
submissions = pd.read_json("submissions.jsonl", lines=True)
comments = pd.read_json("comments.jsonl", lines=True)
print(submissions.shape[0])
print(comments.shape[0])

20
1431


In [ ]:
def form_reddit_threads(submission, comments_df) -> str:
    """
    Print Reddit  submission and their comments in a thread-like structure
    using the parent_id attribute to connect comments to their parents.
    """
    separating_line = "=" * 50

    submission_text = f"{separating_line}\nSUBMISSION: {submission['title']}\n\n{submission['selftext']}\n\nCOMMENTS:\n\n"

    # Get all comments for this submission
    submission_id = f"t3_{submission['id']}" # Reddit prefixes submission IDs with t3_
    submission_comments = comments_df[comments_df['link_id'] == submission_id].copy()

    if len(submission_comments) == 0:
      return submission_text

    # Create a map for faster lookups
    comment_map = {f"t1_{comment['id']}": comment for _, comment in submission_comments.iterrows()}

    # Keep track of processed comments to avoid duplicates
    processed_comments = set()

    # Add a depth attribute to each comment
    submission_comments['depth'] = 0

    # Find top-level comments (those whose parent_id matches the submission_id)
    top_level_comments = submission_comments[submission_comments['parent_id'] == submission_id]

    # For each top-level comment, print it and its replies recursively
    for _, comment in top_level_comments.iterrows():
        submission_text = _add_comment_thread(text=submission_text, comment=comment, comment_map=comment_map, processed_comments=processed_comments, depth=0)

    return submission_text + separating_line

def _add_comment_thread(comment, comment_map, processed_comments, depth=0, text:str|None=None) -> str:
    """Recursively print a comment and its replies with proper indentation"""
    comment_id = f"t1_{comment['id']}"

    # Skip if already processed to avoid duplication
    if comment_id in processed_comments:
        return text

    processed_comments.add(comment_id)

    # Format the comment with appropriate indentation
    indent = "    " * depth

    if not text:
      text = ""

    text += f"{indent}> {comment['body']}\n"

    # Find and print all child comments
    for potential_child_id, potential_child in comment_map.items():
        if potential_child['parent_id'] == comment_id:
            text = _add_comment_thread(text=text, comment=potential_child, comment_map=comment_map, processed_comments=processed_comments, depth= depth + 1)

    return text

## Baseline Strategy: Summarize by submission

In [ ]:
def baseline_summarize_submission(submission:pd.Series, comments:pd.DataFrame) -> str:

  submission_text = form_reddit_threads(submission, comments)

  prompt = ("You are a LLM-powered social listener. Your task is to summarize this reddit submission retrieved from `r/ufc` subreddit. "
  "The topic is how people of the internet think of the fight between Khabib Nurmagomedov vs Conor McGregor\n"
  "Here is the submission:\n\n"
  f"{submission_text}\n\n"
  "Please answer in this format:\n\n"
  "# Submission Summarization\n\n"
  "## Main Summary\n\n"
  "{overall summary of the submission according to the topic.}\n\n"
  "## Summary by aspects\n\n"
  "{list different significant aspects and provide summary of each in this format: `- **aspect**: summary`}\n\n"
  "## Conclusion\n\n"
  "{conclude the entire summary}")
  # print(prompt) #debug

  response = generate_content_with_rate_limit(prompt)
  return response

In [ ]:
test_submission = submissions.iloc[0]
test_summary = baseline_summarize_submission(test_submission, comments)

In [ ]:
from IPython.display import Markdown

# display(Markdown(test_summary.text))
display(Markdown(test_summary))

# Submission Summarization

## Main Summary

The Reddit submission on r/ufc discusses the potential punishment for Khabib Nurmagomedov following his actions after the Conor McGregor fight, with the author arguing that while Khabib's actions were wrong and deserve punishment, the UFC's lenient treatment of Conor McGregor's past transgressions, particularly the bus attack, contributed to the situation. The comments section debates the severity of Khabib's actions, the role of Conor's behavior in instigating the conflict, and the potential consequences, including suspensions, fines, and the stripping of the title. There is also discussion about Dana White's power and potential motivations, as well as the Nevada State Athletic Commission's involvement.

## Summary by aspects

- **Punishment for Khabib**: The main point of discussion is what punishment Khabib should receive, with opinions ranging from severe penalties like stripping the belt to more lenient fines and suspensions.
- **Conor's Role and Past Actions**: A significant portion of the discussion revolves around Conor McGregor's past actions, particularly the bus attack, and whether the UFC's lack of serious punishment for him influenced Khabib's behavior.
- **UFC's Double Standards**: Many users accuse the UFC of having double standards, favoring Conor McGregor due to his popularity and financial draw, and not consistently punishing fighters for their actions.
- **Who is responsible for the punishment**: Some users point out that the Nevada State Athletic Commission, not the UFC, will be the one to decide the punishment.
- **Potential Future Scenarios**: There is speculation about potential future fights, including a rematch between Khabib and Conor, or Khabib facing Tony Ferguson, and how the punishments might affect these scenarios.
- **Staged Fight Theory**: A few users entertain the idea that the post-fight chaos might have been staged to generate hype and increase the financial stakes for a potential rematch.
- **Conor's sucker punch**: Some users point out that Conor sucker punched one of Khabib's teammates inside the cage, which may have triggered Khabib's reaction.

## Conclusion

The Reddit thread reflects a divided opinion on the Khabib-Conor post-fight incident, with many acknowledging Khabib's wrongdoing but also pointing to Conor's past behavior and the UFC's inconsistent disciplinary actions as contributing factors. The discussion encompasses potential punishments, future fight scenarios, and the overall impact on the sport's image.


In [ ]:
def baseline_summarize(submissions:pd.DataFrame, comments:pd.DataFrame) -> str:
  """
  Summarize all submissions in the submissions dataframe
  """
  submission_summaries = {}
  for _, submission in tqdm(submissions.iterrows(), desc="Summarizing submissions", total=submissions.shape[0]):
    summary = baseline_summarize_submission(submission, comments)
    submission_summaries[submission['id']] = summary

  separating_line = ("-" * 50) + "\n"

  prompt = ("You are a LLM-powered social listener. Your task is to summarize these summaries from different reddit submissions retrieved from `r/ufc` subreddit. "
  "The topic is how people of the internet think of the fight between Khabib Nurmagomedov vs Conor McGregor\n"
  "Here are the submission summaries:\n\n"
  f"{separating_line}\n"
  f"{separating_line.join([summary for summary in submission_summaries.values()])}\n\n"
  f"{separating_line}\n"
  "Please answer in this format:\n\n"
  "# Reddit Social Listening Summarization\n\n"
  "## Main Summary\n\n"
  "{overall summary according to the topic.}\n\n"
  "## Summary by aspects\n\n"
  "{list different significant aspects and provide summary of each in this format: `- **aspect**: summary`}\n\n"
  "## Conclusion\n\n"
  "{conclude the entire summary}")
  # print(prompt) #debug

  response = generate_content_with_rate_limit(prompt)
  return (response, submission_summaries)

In [ ]:
baseline_summary, summary_by_submission = baseline_summarize(submissions, comments)
# display(Markdown(baseline_summary.text))
display(Markdown(baseline_summary))

Summarizing submissions:   0%|          | 0/20 [00:00<?, ?it/s]

# Reddit Social Listening Summarization

## Main Summary

The Reddit community on r/ufc is highly engaged and divided in its opinions regarding the Khabib Nurmagomedov vs. Conor McGregor fight and its aftermath. Discussions encompass a wide range of topics, including the fairness of the fight, the post-fight brawl, potential punishments for both fighters, the UFC's handling of the situation, and the possibility of a rematch. A central theme revolves around perceived double standards in the treatment of McGregor versus Khabib, with many questioning the UFC's motivations and the impact of financial incentives on decision-making.

## Summary by aspects

-   **Fairness of the Fight**: Most agree that Khabib won the fight legitimately within the octagon, but some found Khabib's wrestling-heavy style boring.
-   **Khabib's Post-Fight Actions**: Opinions are sharply divided, with some viewing it as an understandable emotional outburst after weeks of trash talk and others seeing it as an unacceptable assault that should be punished. The spitting incident is also a point of contention.
-   **McGregor's Past Behavior**: Many commenters bring up McGregor's history of controversial actions, including the bus incident, and question whether he has faced adequate consequences. His trash talk is debated as "just business" or crossing into personal territory.
-   **UFC's Double Standards**: Many users accuse the UFC of having double standards, favoring Conor McGregor due to his popularity and financial draw, and not consistently punishing fighters for their actions.
-   **Potential Consequences for Khabib**: There's speculation about potential suspensions, fines, and even deportation, with some believing his career is over and others thinking he will face consequences but not be ruined. Some discuss the possibility of Khabib being stripped of his title.
-   **Rematch Debate**: The community is split on whether McGregor deserves a rematch, with many arguing he should prove himself against other contenders first. The financial incentives for the UFC to make it happen are acknowledged.
-   **Khabib's Marketability**: There's a debate on whether Khabib is marketable beyond his fighting skills, particularly in comparison to McGregor. Some believe he is marketable, particularly in Asia and Russia, and that his "no trash-talking" approach can attract a different audience.
-   **Refereeing**: Herb Dean's refereeing is criticized, with many believing he favored McGregor and failed to penalize him for multiple illegal moves.
-   **Racism Allegations**: There is discussion on whether Conor McGregor was truly racist towards Khabib Nurmagomedov during their pre-fight trash talk.
-   **Khabib's Threat to Leave**: Khabib's threat to leave the UFC if his teammate Zubaira Tukhugov is cut is discussed, along with the potential fallout for the organization.

## Conclusion

The Reddit threads reflect a complex and highly polarized view of the Khabib vs. McGregor fight and its aftermath. While Khabib's victory is generally acknowledged, the discussion revolves around the justification, consequences, and broader implications of the post-fight events, with comparisons to McGregor's past actions and the UFC's handling of controversial fighters. The overall sentiment is highly divided, reflecting the complex and emotional nature of the rivalry and the sport itself.


In [ ]:
for id in summary_by_submission: # {submission_id:summary}
  submissions["llm_summary"] = submissions["id"].map(summary_by_submission)

In [ ]:
display(submissions["llm_summary"].describe())

,llm_summary
count,20
unique,20
top,# Submission Summarization\n\n## Main Summary\...
freq,1


In [ ]:
submissions.to_json("submissions_with_llm_summary.jsonl", orient="records", lines=True)

In [ ]:
with open("llm_top_level_summary.md", mode="w") as textfile:
  textfile.write(baseline_summary)

## Comment Classification

In [8]:
def comment_context(target_comment: pd.Series, comments: pd.DataFrame):
    """
    Create context string in submission and thread structure for a given comment.
    """
    separating_line = "=" * 50

    # Extract the submission ID from the comment's link_id
    submission_id = target_comment['link_id']  # This should be in format 't3_xyz'

    # If the submission ID doesn't start with 't3_', prepare it
    if not submission_id.startswith('t3_'):
        submission_id = f"t3_{submission_id}"

    # Find the submission from its ID
    # Note: This assumes there's a 'submissions' DataFrame available
    # You'll need to adjust this to match how you're accessing submissions
    submission_raw_id = submission_id.replace('t3_', '')
    submission = submissions[submissions['id'] == submission_raw_id].iloc[0]

    # Build the context with the submission
    context = f"{separating_line}\nSUBMISSION: {submission['title']}\n\n{submission['selftext']}\n\nCOMMENTS:\n\n"
    # if pd.notna(submission['selftext']):
    #     context += f"{submission['selftext']}\n\n"

    # Get the comment's parent comments to build the thread context
    parent_comments = []
    current_comment = target_comment

    # Traverse up the parent chain until we reach the submission
    while True:
        parent_id = current_comment['parent_id']

        # If parent is the submission, we're done
        if parent_id == submission_id:
            break

        # Find the parent comment
        parent_raw_id = parent_id.replace('t1_', '')
        parent_comments_filtered = comments[comments['id'] == parent_raw_id]

        if parent_comments_filtered.empty:
            break  # Parent comment not found

        parent_comment = parent_comments_filtered.iloc[0]
        parent_comments.append(parent_comment)
        current_comment = parent_comment

    # Reverse the list to get from top to bottom order
    parent_comments.reverse()

    # Add all parent comments to the context with proper indentation
    for i, comment in enumerate(parent_comments):
        indent = "    " * i
        context += f"{indent}> {comment['body']}\n\n"

    return context + separating_line

def zero_shot_llm_sentiment_classify(submissions: pd.DataFrame, comments: pd.DataFrame, sentiments: list):
    """
    Iterate on each comment and use corresponding submission and parent comments as context
    to classify the sentiment.
    """
    separating_line = ("-" * 50) + "\n"
    results = {}

    try:
      # Process each comment
      for _, comment in tqdm(comments.iterrows(), desc="Classifying comments", total=comments.shape[0]):
          # Get context for this comment
          context = comment_context(comment, comments)

          # Construct the prompt
          prompt = ("You are a context-aware comment text classifier. "
                    f"Your task is to classify a given comment in these sentiments: {', '.join(sentiments[:-1])}, and {sentiments[-1]}. "
                    "The sentiment must be considered towards the fight, not preceding comments, although this can be considered as well.\n\n"
                    "Use the submission and, if any, parent comments as context. Here is the context:\n\n"
                    f"{separating_line}\n"
                    f"{context}\n"
                    f"{separating_line}\n"
                    "Here is the comment:\n\n"
                    f"{separating_line}\n"
                    f"{comment['body']}\n"
                    f"{separating_line}\n"
                    "Please provide reasoning and label in this format:\n\n"
                    "Reasoning: {reasoning}\n"
                    "Label: {label}"
                    )

          # Get LLM response
          response = generate_content_with_rate_limit(prompt)

          # Extract the sentiment label
          # Assuming the response follows the format with a label at the end
          try:
              reasoning = response.split("Reasoning: ")[1].split("Label: ")[0].strip()
              print(f"reasoning: {reasoning}")
              label = response.split("Label: ")[1].strip().lower()
              print(f"label: {label}")
          except IndexError:
              # If parsing fails, store the raw response as reasoning and mark label as unknown
              label_part = "unknown"

          # Store the result
          results[comment['id']] = label

    except KeyboardInterrupt:
      print("Interrupted by user")
    finally:
      return results

In [9]:
sentiment_classified = zero_shot_llm_sentiment_classify(submissions, comments, ["positive", "negative", "neutral"])

Classifying comments:   0%|          | 0/1431 [00:00<?, ?it/s]

reasoning: The comment is a reaction to the length of the submission, not the fight itself or the actions of the fighters. It's a neutral observation about the timing of the post.
label: neutral
reasoning: The comment defends Khabib's actions by pointing out that Conor has also jumped out of the cage to confront people. This suggests a negative sentiment towards the punishment Khabib is facing and a defense of his actions.
label: negative
reasoning: The comment agrees with the submission's point that Conor's actions and the UFC's lack of punishment for them influenced Khabib's behavior. It expresses a negative sentiment towards Conor and the UFC.
label: negative
reasoning: The comment expresses amusement at the idea that Khabib cannot be stripped of his title, directly contradicting that notion. This suggests a negative sentiment towards Khabib's actions and a belief that he should be punished.
label: negative
reasoning: The comment expresses disapproval of Khabib's actions after the f

reasoning: The comment acknowledges the previous discussion about Khabib's actions and punishment, indicating a neutral stance towards the fight itself.
label: neutral
reasoning: The comment discusses the possibility of Khabib being stripped of his title and how the UFC might handle the situation, suggesting a fight between the number 2 and 3 lightweights for the vacant title. This is a neutral observation about potential consequences.
label: neutral
reasoning: The comment agrees with the parent comment and discusses potential scenarios for the title shot, including Conor getting an immediate shot, Nate vs. Poirier winner being a contender, and Tony Ferguson being involved regardless of Khabib's status. It doesn't express strong positive or negative sentiment towards the fight itself, but rather discusses potential outcomes and contenders.
label: neutral
reasoning: The comment expresses strong negative sentiment towards both fighters, advocating for their removal from the UFC due to th

reasoning: The comment provides evidence of Khabib throwing a punch, which is relevant to the discussion of his punishment. It doesn't express a clear positive, negative, or neutral sentiment towards the fight itself, but rather provides factual information.
label: neutral
reasoning: The comment is questioning the authenticity of the events surrounding Khabib's punishment, suggesting it might have been staged. This doesn't express a positive or negative sentiment towards the fight or the fighters involved, but rather a skeptical perspective on the situation.
label: neutral
reasoning: The comment "Even better than" is incomplete and lacks context to determine sentiment towards the fight or the fighters involved. It's a fragment of a thought and doesn't express a clear opinion.
label: neutral
reasoning: The comment expresses a negative sentiment towards Conor McGregor, referencing a past incident where he allegedly assaulted an official. It is relevant to the discussion of punishment and

reasoning: The comment expresses disagreement with the previous comment's assertion that there will not be a Khabib vs. Conor 2 fight. It questions the source of that information and suggests that the fight is likely to happen in the future. The comment does not express strong positive or negative sentiment towards either fighter or the fight itself, but rather a neutral stance of disagreement and prediction.
label: neutral
reasoning: The comment expresses a desire to see Khabib fight Tony for the belt, which implies a positive sentiment towards Khabib retaining the belt and continuing to fight.
label: positive
reasoning: The comment "It could win you a presidency" is a sarcastic remark about the potential benefits of controversial actions, likely referring to Khabib's actions and the attention they garnered. It implies that such behavior, while wrong, could be seen as a display of strength or defiance, potentially appealing to certain audiences. This is a negative sentiment towards th

...

reasoning: The comment agrees with the previous comment about Khabib's stock rising, but then argues that Khabib is still bound by his contract and the UFC ultimately controls the opponent selection. The comment also includes a quote from Khabib himself dismissing the $15 million offer as "bullshit news". The comment is mostly neutral, discussing the business aspects of fighting and contracts.
label: neutral
reasoning: The comment is complaining about the upvotes and downvotes, which is not directly related to the fight itself or the fighters. It's a meta-comment about the voting system on the platform.
label: neutral
reasoning: The comment suggests that Conor would lose if he doesn't fully commit to the fight, regardless of the opponent. This implies a negative sentiment towards Conor's chances in a fight.
label: negative
reasoning: The comment expresses disbelief and accuses the previous commenter of being delusional for believing the claim that Khabib turned down $15 million to figh

reasoning: The comment defends McGregor's actions in the fight, arguing that his movement was a smart tactic and not running away. It highlights McGregor's success in the fight, mentioning he dropped Nate multiple times. This suggests a positive sentiment towards McGregor's performance.
label: positive


reasoning: The comment expresses agreement with the preceding comment, which defends McGregor's performance in the second fight against Diaz and argues that Diaz didn't "destroy" or "dominate" McGregor. This implies a positive sentiment towards McGregor's chances in a potential third fight.
label: positive


reasoning: The comment expresses a strong opinion that Nate Diaz was unfairly treated ("robbed") and that he dominated Conor McGregor ("smacked Conor up"). It also provides a specific reason for why Diaz was dropped, suggesting a strategic choice rather than a sign of weakness. This indicates a negative sentiment towards the fight's outcome and a positive sentiment towards Diaz's performance.
label: negative
reasoning: The comment acknowledges the previous discussion about the fight and then states a fact about the fight. It doesn't express a strong positive or negative sentiment towards either fighter, but rather focuses on a specific event in the fight.
label: neutral
reasoning: The comment expresses a positive sentiment towards Nate Diaz's performance in the second fight, implying he was winning or doing well in that round.
label: positive
reasoning: The comment is arguing that McGregor was running away from Diaz out of fear, not strategy. This is a negative sentiment towards McGreg

reasoning: The comment expresses a negative sentiment towards a potential fight between Silva and Conor, calling it a "stupid matchup." It then suggests an alternative fight (Conor vs. Pettis) and a potential future fight (Conor vs. Tony) that it considers a "true, great matchup." The comment is therefore negative towards the proposed fight in the submission.
label: negative
reasoning: The comment is discussing Nate Diaz's physique leading up to a fight, implying he might be in good shape for a potential McGregor fight. This suggests a positive outlook on Diaz's chances, and therefore, a more competitive and exciting fight.
label: positive
reasoning: The comment expresses concern about McGregor's future fights, particularly against Silva, and analyzes the potential Diaz fight. It acknowledges McGregor's past success but also points out his recent struggles and the fading of his "mysticism." The overall sentiment is mixed, but leans towards a critical assessment of McGregor's current si

In [10]:
for id in sentiment_classified:
  comments["llm_sentiment"] = comments["id"].map(sentiment_classified)

In [11]:
comments.to_json("comments_with_llm_sentiment.jsonl", orient="records", lines=True)

In [16]:
mixed = {id: sentiment for id, sentiment in sentiment_classified.items() if sentiment == "mixed"}

In [17]:
len(mixed)

3

In [15]:
display(set(comments["llm_sentiment"].tolist())) # somehow another sentiment was added

{'mixed', 'negative', 'neutral', 'positive'}